# ETL Project - Portfolio Management Data

The customer currently manages their portfolio of projects using Excel spreadsheets and it is difficult for them to know if or how each project is tied to the company’s strategies. The customer has provided two separate Excel spreadsheets and I saved them as CSV files: (1) project listing with project details and (2) project strategy and risk information. 

Please be aware that this is not proprietary data. The customer provided sample data and they had modified anything that was proprietary before turning over the Excel files.

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
from config import db_connection

# Read the CSV files
projects = "Data/projects.csv"
strategy = "Data/strategy.csv"

In [2]:
# Create the data frame - projects
projects_df = pd.read_csv(projects, delimiter=',', skipinitialspace=True)

# Display the projects data for preview
projects_df

,Project ID,Project Name,Start Date,End Date,Days,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Life Cycle,Project Type,Status,Funding Status,Type,Total Actual Cost
0,AB-U38510550,Upgrd - Facilities Maintenance System,6/7/2019,5/1/2020,329.0,Managing assets is a critical component to the...,4,"75,497","163,000","Greiff, Gene",Annuities,Operate,Breakthrough,Open,Finished,Six Sigma Kaizen event,"75,497"
1,AC-D39121594,DB2 to Oracle Migration,2/8/2018,11/10/2018,275.0,NaN,NaN,"310,000","310,000","Kellerman, Henry",Asset Management,Business Case,Core,Open,Funded,IT Waterfall traditional implementation,0
2,AC-G38719560,General Ledger (ERP),1/2/2020,4/20/2020,109.0,The General Ledger (ERP) project supports the ...,?,0,"550,000","Mitchell, Linda",Asset Management,Delivery,Core,Open,Proposed,IT Waterfall traditional implementation,0
3,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,324.0,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Concept,Core,Open,Funded,Six Sigma Kaizen event,0
4,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,324.0,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Concept,Core,Open,Funded,Six Sigma Kaizen event,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,WS-S40909321,Single Sign On and Portal,1/1/2020,12/31/2020,365.0,This project is intended to provide a single s...,2,NaN,"115,500","Waterman, Lee",Wholesale,Initiation,Strategic Priority,Candidate,Demand,Six Sigma Kaizen event,NaN
631,WS-S40909791,Sales Training Design,1/1/2020,1/1/2020,0.0,Implement RM on-boarding and training program ...,?,NaN,"142,300","Rink, Jenna",Wholesale,Initiation,Discretionary,Candidate,Demand,Six Sigma Kaizen event,NaN
632,WS-T39914244,Treasury Portal,4/11/2020,11/15/2020,218.0,Treasury clients today use a variety of produc...,4,0,"156,000",NaN,Wholesale,Execution,Breakthrough,Open,Canceled,IT Waterfall traditional implementation,0
633,WS-W40909395,Workflow Enablement - Wave 3,1/1/2020,1/1/2020,0.0,Extend workflow capabilities to all LOBs,3,NaN,"101,000","Street, Stella",Wholesale,Initiation,Improvement/Line Extension,Candidate,Demand,Six Sigma Kaizen event,0


In [3]:
# Create the data frame - strategy
strategy_df = pd.read_csv(strategy, delimiter=',', skipinitialspace=True)

# Display the strategy data for preview
strategy_df

,Project ID,Strategy Score,Risk Score,Risk - # of Vendors Contributing,Risk - Business Problem Clearly Defined,Risk - Business Processes Impacted,Risk - Impact to Key Financial Application,Risk - Level of User Change Management,Risk - Market,"Risk - New Product, Functionality or Service",...,Support for Competitive Differentiation,Mandatory?,Business Priority,Forced Rank,Risk Eval,Risk - Acceptance,Risk - Skills / Capability,Risk - Complexity,Risk - Product / Technology,Risk - Planning
0,AB-U38510550,59,73.0,High,Med,Med,Low,High,Med,Low,...,Very Strong,No,NaN,NaN,67.0,Med,Med,High,Low,Med
1,AC-D39121594,36,33.0,Low,Low,Low,Low,Low,Low,Low,...,Moderate,NaN,NaN,NaN,33.0,Low,Low,Low,Low,Low
2,AC-G38719560,36,77.0,Med,Low,Med,High,Med,High,High,...,Weak,No,NaN,NaN,83.0,High,Med,Med,High,Low
3,AC-M39177596,50,40.0,Low,Low,Low,Low,Low,Med,Low,...,Strong,NaN,NaN,NaN,50.0,Med,Low,Low,Low,Low
4,AC-R38763567,89,70.0,Med,High,Med,High,Med,Low,High,...,Very Strong,NaN,NaN,NaN,50.0,Low,Med,Med,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,WS-S40909321,75,43.0,Low,Low,Med,Med,Low,Low,Med,...,Moderate,No,NaN,NaN,50.0,Low,Med,Low,Med,Low
610,WS-S40909791,52,43.0,Low,Low,Med,Med,Low,Low,Med,...,Moderate,No,NaN,NaN,50.0,Low,Med,Low,Med,Low
611,WS-T39914244,66,70.0,Med,High,Med,High,Med,Low,High,...,Very Strong,NaN,NaN,NaN,50.0,Low,Med,Med,High,High
612,WS-W40909395,37,57.0,Low,Med,High,Med,Low,Med,Med,...,Weak,Yes,NaN,NaN,83.0,Med,High,Low,Med,Med


## Transform the data

* Remove duplicates, select columns, clean data, join the two datasets

In [4]:
# Check the number of unique records in projects using groupby
projects_group = projects_df.groupby(["Project ID"])
len(projects_group)

629

In [5]:
# Get all the data for the duplicate rows in projects
duplicateRow = projects_df[projects_df.duplicated()]
duplicateRow

,Project ID,Project Name,Start Date,End Date,Days,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Life Cycle,Project Type,Status,Funding Status,Type,Total Actual Cost
4,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,324.0,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Concept,Core,Open,Funded,Six Sigma Kaizen event,0
6,AC-R38763567,Requirements System,2/15/2020,3/23/2020,37.0,NaN,NaN,"186,000","186,000","Vincent, Jeanine",Annuities,Concept,Core,Open,Funded,IT Waterfall traditional implementation,0
426,TM-L39909243,Legacy MDS Migration,4/6/2020,6/15/2020,70.0,Update the middleware Message Broker flows fro...,?,"159,000","155,250",NaN,Technology and Operations,Realization,Maintenance-Other,Open,Completed,Six Sigma Kaizen event,NaN
495,WM-D40909401,Deliver ESP Card Releases,1/1/2020,12/31/2020,365.0,Focused on enhancing the competitiveness of th...,5,NaN,"83,781","Hammond, Carrie",Wholesale,Initiation,Maintenance-Other,Candidate,Canceled,IT Waterfall traditional implementation,0
526,WM-T4090977,TPS EIS Planning Allocation,1/1/2020,12/31/2020,365.0,TPS EIS Planning Allocation,0,NaN,"65,152",NaN,Wholesale,Initiation,Discretionary,Candidate,Canceled,IT Waterfall traditional implementation,0


In [6]:
# List the index for each duplicated rows in projects
list_index = projects_df[projects_df.duplicated()].index.tolist()
list_index

[4, 6, 426, 495, 526]

In [7]:
# Create a clean projects data frame by dropping the duplicate rows by its index
projects_df = projects_df.drop(list_index)
projects_df

,Project ID,Project Name,Start Date,End Date,Days,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Life Cycle,Project Type,Status,Funding Status,Type,Total Actual Cost
0,AB-U38510550,Upgrd - Facilities Maintenance System,6/7/2019,5/1/2020,329.0,Managing assets is a critical component to the...,4,"75,497","163,000","Greiff, Gene",Annuities,Operate,Breakthrough,Open,Finished,Six Sigma Kaizen event,"75,497"
1,AC-D39121594,DB2 to Oracle Migration,2/8/2018,11/10/2018,275.0,NaN,NaN,"310,000","310,000","Kellerman, Henry",Asset Management,Business Case,Core,Open,Funded,IT Waterfall traditional implementation,0
2,AC-G38719560,General Ledger (ERP),1/2/2020,4/20/2020,109.0,The General Ledger (ERP) project supports the ...,?,0,"550,000","Mitchell, Linda",Asset Management,Delivery,Core,Open,Proposed,IT Waterfall traditional implementation,0
3,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,324.0,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Concept,Core,Open,Funded,Six Sigma Kaizen event,0
5,AC-R38763567,Requirements System,2/15/2020,3/23/2020,37.0,NaN,NaN,"186,000","186,000","Vincent, Jeanine",Annuities,Concept,Core,Open,Funded,IT Waterfall traditional implementation,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,WS-S40909321,Single Sign On and Portal,1/1/2020,12/31/2020,365.0,This project is intended to provide a single s...,2,NaN,"115,500","Waterman, Lee",Wholesale,Initiation,Strategic Priority,Candidate,Demand,Six Sigma Kaizen event,NaN
631,WS-S40909791,Sales Training Design,1/1/2020,1/1/2020,0.0,Implement RM on-boarding and training program ...,?,NaN,"142,300","Rink, Jenna",Wholesale,Initiation,Discretionary,Candidate,Demand,Six Sigma Kaizen event,NaN
632,WS-T39914244,Treasury Portal,4/11/2020,11/15/2020,218.0,Treasury clients today use a variety of produc...,4,0,"156,000",NaN,Wholesale,Execution,Breakthrough,Open,Canceled,IT Waterfall traditional implementation,0
633,WS-W40909395,Workflow Enablement - Wave 3,1/1/2020,1/1/2020,0.0,Extend workflow capabilities to all LOBs,3,NaN,"101,000","Street, Stella",Wholesale,Initiation,Improvement/Line Extension,Candidate,Demand,Six Sigma Kaizen event,0


In [8]:
# Select the project columns needed for portfolio analysis
projects_df = projects_df[["Project ID", "Project Name","Start Date", "End Date", "Description",
                           "Number of Dependents", "Total Funded Budget", "Total Planned Cost", 
                           "Portfolio Executive", "Segment", "Project Type", "Status", "Funding Status", 
                           "Total Actual Cost"]]
                     
# Display the new dataframe
projects_df.head()

,Project ID,Project Name,Start Date,End Date,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Project Type,Status,Funding Status,Total Actual Cost
0,AB-U38510550,Upgrd - Facilities Maintenance System,6/7/2019,5/1/2020,Managing assets is a critical component to the...,4,"75,497","163,000","Greiff, Gene",Annuities,Breakthrough,Open,Finished,"75,497"
1,AC-D39121594,DB2 to Oracle Migration,2/8/2018,11/10/2018,NaN,NaN,"310,000","310,000","Kellerman, Henry",Asset Management,Core,Open,Funded,0
2,AC-G38719560,General Ledger (ERP),1/2/2020,4/20/2020,The General Ledger (ERP) project supports the ...,?,0,"550,000","Mitchell, Linda",Asset Management,Core,Open,Proposed,0
3,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Core,Open,Funded,0
5,AC-R38763567,Requirements System,2/15/2020,3/23/2020,NaN,NaN,"186,000","186,000","Vincent, Jeanine",Annuities,Core,Open,Funded,0


In [9]:
# Check the number of unique records in strategy using groupby
strategy_group = strategy_df.groupby(["Project ID"])
len(strategy_group)

612

In [10]:
# Get all the data for the duplicate rows in strategy
duplicateRow = strategy_df[strategy_df.duplicated()]
duplicateRow

,Project ID,Strategy Score,Risk Score,Risk - # of Vendors Contributing,Risk - Business Problem Clearly Defined,Risk - Business Processes Impacted,Risk - Impact to Key Financial Application,Risk - Level of User Change Management,Risk - Market,"Risk - New Product, Functionality or Service",...,Support for Competitive Differentiation,Mandatory?,Business Priority,Forced Rank,Risk Eval,Risk - Acceptance,Risk - Skills / Capability,Risk - Complexity,Risk - Product / Technology,Risk - Planning
412,TM-F40287631,76,50.0,Low,Med,High,Med,Low,Low,Med,...,Weak,No,NaN,NaN,67.0,Low,High,Low,Med,Med
506,WM-R40940455,73,33.0,Low,Low,Low,Low,Low,Low,Low,...,Mandatory,Yes,NaN,NaN,33.0,Low,Low,Low,Low,Low


In [11]:
# List the index for each duplicated rows in strategy
list_index = strategy_df[strategy_df.duplicated()].index.tolist()
list_index

[412, 506]

In [12]:
# Create a clean strategy data frame by dropping the duplicate rows by its index
strategy_df = strategy_df.drop(list_index)
strategy_df

,Project ID,Strategy Score,Risk Score,Risk - # of Vendors Contributing,Risk - Business Problem Clearly Defined,Risk - Business Processes Impacted,Risk - Impact to Key Financial Application,Risk - Level of User Change Management,Risk - Market,"Risk - New Product, Functionality or Service",...,Support for Competitive Differentiation,Mandatory?,Business Priority,Forced Rank,Risk Eval,Risk - Acceptance,Risk - Skills / Capability,Risk - Complexity,Risk - Product / Technology,Risk - Planning
0,AB-U38510550,59,73.0,High,Med,Med,Low,High,Med,Low,...,Very Strong,No,NaN,NaN,67.0,Med,Med,High,Low,Med
1,AC-D39121594,36,33.0,Low,Low,Low,Low,Low,Low,Low,...,Moderate,NaN,NaN,NaN,33.0,Low,Low,Low,Low,Low
2,AC-G38719560,36,77.0,Med,Low,Med,High,Med,High,High,...,Weak,No,NaN,NaN,83.0,High,Med,Med,High,Low
3,AC-M39177596,50,40.0,Low,Low,Low,Low,Low,Med,Low,...,Strong,NaN,NaN,NaN,50.0,Med,Low,Low,Low,Low
4,AC-R38763567,89,70.0,Med,High,Med,High,Med,Low,High,...,Very Strong,NaN,NaN,NaN,50.0,Low,Med,Med,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,WS-S40909321,75,43.0,Low,Low,Med,Med,Low,Low,Med,...,Moderate,No,NaN,NaN,50.0,Low,Med,Low,Med,Low
610,WS-S40909791,52,43.0,Low,Low,Med,Med,Low,Low,Med,...,Moderate,No,NaN,NaN,50.0,Low,Med,Low,Med,Low
611,WS-T39914244,66,70.0,Med,High,Med,High,Med,Low,High,...,Very Strong,NaN,NaN,NaN,50.0,Low,Med,Med,High,High
612,WS-W40909395,37,57.0,Low,Med,High,Med,Low,Med,Med,...,Weak,Yes,NaN,NaN,83.0,Med,High,Low,Med,Med


In [13]:
# Select the strategy and risk columns needed for the analysis
strategy_df = strategy_df[["Project ID", "Strategy Score","Primary Support Strategy", "Risk Score"]]
                     
# dDisplay the new data frame
strategy_df

,Project ID,Strategy Score,Primary Support Strategy,Risk Score
0,AB-U38510550,59,Support for Competitive Differentiation,73.0
1,AC-D39121594,36,Support for External Customers,33.0
2,AC-G38719560,36,Support for Increased Productivity,77.0
3,AC-M39177596,50,Support for External Customers,40.0
4,AC-R38763567,89,Support for External Customers,70.0
...,...,...,...,...
609,WS-S40909321,75,Support for Increased Productivity,43.0
610,WS-S40909791,52,Support for Increased Productivity,43.0
611,WS-T39914244,66,Support for External Customers,70.0
612,WS-W40909395,37,Support for Reducing Costs,57.0


### Replacing missing values in projects with 0

* Some of the variables have a value of "?" and need to be replaced with a zero

In [14]:
# Replacing missing data with a zero
projects_df = projects_df.replace(to_replace ="?", 
                                 value =0) 

In [15]:
# Preview clean dataset
projects_df

,Project ID,Project Name,Start Date,End Date,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Project Type,Status,Funding Status,Total Actual Cost
0,AB-U38510550,Upgrd - Facilities Maintenance System,6/7/2019,5/1/2020,Managing assets is a critical component to the...,4,"75,497","163,000","Greiff, Gene",Annuities,Breakthrough,Open,Finished,"75,497"
1,AC-D39121594,DB2 to Oracle Migration,2/8/2018,11/10/2018,NaN,NaN,"310,000","310,000","Kellerman, Henry",Asset Management,Core,Open,Funded,0
2,AC-G38719560,General Ledger (ERP),1/2/2020,4/20/2020,The General Ledger (ERP) project supports the ...,0,0,"550,000","Mitchell, Linda",Asset Management,Core,Open,Proposed,0
3,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Core,Open,Funded,0
5,AC-R38763567,Requirements System,2/15/2020,3/23/2020,NaN,NaN,"186,000","186,000","Vincent, Jeanine",Annuities,Core,Open,Funded,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,WS-S40909321,Single Sign On and Portal,1/1/2020,12/31/2020,This project is intended to provide a single s...,2,NaN,"115,500","Waterman, Lee",Wholesale,Strategic Priority,Candidate,Demand,NaN
631,WS-S40909791,Sales Training Design,1/1/2020,1/1/2020,Implement RM on-boarding and training program ...,0,NaN,"142,300","Rink, Jenna",Wholesale,Discretionary,Candidate,Demand,NaN
632,WS-T39914244,Treasury Portal,4/11/2020,11/15/2020,Treasury clients today use a variety of produc...,4,0,"156,000",NaN,Wholesale,Breakthrough,Open,Canceled,0
633,WS-W40909395,Workflow Enablement - Wave 3,1/1/2020,1/1/2020,Extend workflow capabilities to all LOBs,3,NaN,"101,000","Street, Stella",Wholesale,Improvement/Line Extension,Candidate,Demand,0


### Merge the two data frames to create the portfolio of projects

In [16]:
# Merge the two DataFrames using a left join
portfolio_df = pd.merge(projects_df, strategy_df, on="Project ID", how="left")
portfolio_df

,Project ID,Project Name,Start Date,End Date,Description,Number of Dependents,Total Funded Budget,Total Planned Cost,Portfolio Executive,Segment,Project Type,Status,Funding Status,Total Actual Cost,Strategy Score,Primary Support Strategy,Risk Score
0,AB-U38510550,Upgrd - Facilities Maintenance System,6/7/2019,5/1/2020,Managing assets is a critical component to the...,4,"75,497","163,000","Greiff, Gene",Annuities,Breakthrough,Open,Finished,"75,497",59.0,Support for Competitive Differentiation,73.0
1,AC-D39121594,DB2 to Oracle Migration,2/8/2018,11/10/2018,NaN,NaN,"310,000","310,000","Kellerman, Henry",Asset Management,Core,Open,Funded,0,36.0,Support for External Customers,33.0
2,AC-G38719560,General Ledger (ERP),1/2/2020,4/20/2020,The General Ledger (ERP) project supports the ...,0,0,"550,000","Mitchell, Linda",Asset Management,Core,Open,Proposed,0,36.0,Support for Increased Productivity,77.0
3,AC-M39177596,Marketing Data Integration,4/5/2018,2/23/2019,NaN,NaN,"150,000","150,000","Bing, Jeremy",Advice & Wealth Management,Core,Open,Funded,0,50.0,Support for External Customers,40.0
4,AC-R38763567,Requirements System,2/15/2020,3/23/2020,NaN,NaN,"186,000","186,000","Vincent, Jeanine",Annuities,Core,Open,Funded,0,89.0,Support for External Customers,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,WS-S40909321,Single Sign On and Portal,1/1/2020,12/31/2020,This project is intended to provide a single s...,2,NaN,"115,500","Waterman, Lee",Wholesale,Strategic Priority,Candidate,Demand,NaN,75.0,Support for Increased Productivity,43.0
626,WS-S40909791,Sales Training Design,1/1/2020,1/1/2020,Implement RM on-boarding and training program ...,0,NaN,"142,300","Rink, Jenna",Wholesale,Discretionary,Candidate,Demand,NaN,52.0,Support for Increased Productivity,43.0
627,WS-T39914244,Treasury Portal,4/11/2020,11/15/2020,Treasury clients today use a variety of produc...,4,0,"156,000",NaN,Wholesale,Breakthrough,Open,Canceled,0,66.0,Support for External Customers,70.0
628,WS-W40909395,Workflow Enablement - Wave 3,1/1/2020,1/1/2020,Extend workflow capabilities to all LOBs,3,NaN,"101,000","Street, Stella",Wholesale,Improvement/Line Extension,Candidate,Demand,0,37.0,Support for Reducing Costs,57.0


In [17]:
# Output data frame to csv
portfolio_df.to_csv("Output/project_portfolio.csv",index=False)

### Project Type

In [18]:
# Overview of Project Type
portfolio_df["Project Type"].value_counts()

Discretionary                 87
New Product                   86
Strategic Priority            83
Regulatory                    81
Maintenance-Other             62
Core                          48
Improvement/Line Extension    43
Breakthrough                  41
Customization                 41
Productivity                  25
Not Aligned                   15
Maintenance                   10
Strategic Initiative           1
Name: Project Type, dtype: int64

### Primary Support Strategy

In [19]:
# Overview of Primary Support Strategy
portfolio_df["Primary Support Strategy"].value_counts()

Support for Reducing Costs                 166
Support for External Customers             158
Support for Competitive Differentiation    102
Support for Increased Productivity          93
Support for Internal Customers              80
Name: Primary Support Strategy, dtype: int64

### Segment

In [20]:
# Overview of Segment
portfolio_df["Segment"].value_counts()

Consumer                      232
Wholesale                     157
Technology and Operations      57
Corporate Functions            53
Risk                           48
Mortgage                       23
Business Intelligence          18
Advice & Wealth Management     11
Asset Management                9
Protection                      8
Annuities                       5
Corporate & Other               5
Not Aligned                     3
Name: Segment, dtype: int64

### Funding Status

In [21]:
# Overview of Funding Status
portfolio_df["Funding Status"].value_counts()

Approved                         170
Demand                           168
Concluded (FOR EPMO USE ONLY)     84
Active                            75
Canceled                          37
New                               32
Funded                            30
On Hold                           14
Completed                          7
Withdrawn (FOR EPMO USE ONLY)      6
Proposed                           6
Finished                           1
Name: Funding Status, dtype: int64

## Load the data

* Load the portfolio data into PostgreSQL database

### Create database connection to accolade_db

In [29]:
# Please verify that db_connection string that is located in config.py is correct before running this cell
engine = create_engine(db_connection)

In [30]:
# Confirm tables
engine.table_names()

['project_portfolio']

In [31]:
# Load data frame into database
portfolio_df.to_sql(name='project_portfolio', con=engine, if_exists='replace')

In [32]:
# Query database to find states with negative energy difference values
strategy_over90 = engine.execute('select "Project ID" from project_portfolio where "Strategy Score" > 90').fetchall()
strategy_over90 = [project_id[0] for project_id in strategy_over90]

# Output query results
print('The following Projects have the highest strategy scores in the portfolio:')
for project_id in strategy_over90:
    print(project_id)

The following Projects have the highest strategy scores in the portfolio:
CM-E40910804
CS-C40358641
CS-C40603685
CS-F40581683
CS-I40755707
CS-I40802301
CS-O40909775
CS-O40909785
CS-P40909772
CS-R4054421
CS-S40483656
MM-R4090968
M-P40940144
RR-F40911136
TR-I41183492
WM-S4090986
WM-D41001169
WM-U42019549
WP-R40770714
WS-E40617695
